# 1_Basics.ipynb

**Exercise 1**
```python
src, dst = list(zip(*edge_list))
G.add_edges(src, dst)
```

**Exercise 2**
```python
G.edata['broness'] += nd.random.randn(G.number_of_edges()) * 0.1
```


# 2_MessagePassing.ipynb
**Exercise 1**
```python
while num_invited < 34:
    G.update_all(message_func, reduce_func)
    num_invited = int(G.ndata['invited'].sum().asscalar())
    print("{} members invited".format(num_invited))
```

# 3_GCN.ipynb
**Exercise 1**
```python
class GCN(gluon.Block):
    def __init__(self, out_feats):
        super(GCN, self).__init__()
        self.linear = nn.Dense(out_feats)
    
    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first perform linear transformation
        h = self.linear(inputs)
        # set the node features
        g.ndata['h'] = h
        # set node degrees
        g.ndata['deg'] = g.in_degrees(g.nodes()).reshape(-1, 1).astype('float32')
        # trigger message passing, gcn_message and gcn_reduce will be defined later
        g.update_all(gcn_message, gcn_reduce)
        # get the result node features
        h = g.ndata.pop('h')
        return h

def gcn_message(edges):
    return {'msg': nd.broadcast_div(edges.src['h'], nd.sqrt(edges.src['deg'] * edges.dst['deg']))}

# Define a 2-layer GCN model
class Model(gluon.Block):
    def __init__(self, hidden_size, num_classes):
        super(Model, self).__init__()
        self.gcn1 = GCN(hidden_size)
        self.gcn2 = GCN(num_classes)
    
    def forward(self, g, inputs):
        h = self.gcn1(g, inputs)
        h = nd.relu(h)
        h = self.gcn2(g, h)
        return h

G = create_karate_graph()
GG = convert_to_bidirectional(G)
inputs = nd.eye(34)  # featureless inputs
labeled_nodes = nd.array([0, 33], dtype=np.int64)  # only the instructor and the president nodes are labeled
labels = nd.array([0, 1], dtype=np.int64)  # their labels are different
model = Model(5, 2)
model.initialize()
loss_fn = gluon.loss.SoftmaxCELoss()
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.05})

all_logits = []
for epoch in range(30):
    with mxnet.autograd.record():
        logits = model(GG, inputs)
        loss = loss_fn(logits[labeled_nodes], labels).sum()
    
    loss.backward()
    trainer.step(batch_size=1)
    
    all_logits.append(logits.detach())
    print('Epoch %d | Loss: %.4f' % (epoch, loss.asscalar()))
```